# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import datetime

#### Creating list of filepaths to process original event csv data files

In [29]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/meep/udacity/dataEngineering/Projects/project_2/Udacity-Data-Modeling-with-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [30]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [31]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [32]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [33]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS songs
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)


#### Set Keyspace

In [34]:
try:
    session.set_keyspace('songs')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## session_tracking_Based (Question 1)
### Example Query:
> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
### Partition Key
- sessionId is used for Parition Key so that the table is distributed based on each session.
### Primary Key
- sessionId itself will not be unique, so that sessionId and itemInSession will be used together to give the unique for each rows.


In [35]:
query = "CREATE TABLE IF NOT EXISTS session_tracking_Based"
query += """
(
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length double,
    PRIMARY KEY(sessionId,itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)


In [36]:
# Load Data to table session_tracking_Based 
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO session_tracking_Based (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


#### Do a SELECT to verify that the data have been inserted into each table

In [37]:
# Support Query Example 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = """
SELECT artist, song, length 
FROM session_tracking_Based
WHERE sessionId = 338 AND itemInSession = 4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)


Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


## user_session_Based (Question 2)
### Example Query:
> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### Partition Key
- `userID` and `sessionId` is used for Parition Key so that the table is distributed based on userID and  session.
### Primary Key
- as the requirement state that we want to filter not only on `userId` but also `sessionId` and ordering on `itemInSession`, so the following 2 keys will also be part of the PK.
> 

In [38]:
## Support Query Example 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_session_Based"
query += """
(
    userID int,
    sessionID int,
    itemInSession int,

    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY((userID, sessionID), itemInSession)
)
WITH CLUSTERING ORDER BY (itemInSession ASC);

"""
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [39]:
# Load Data to table user_Session_based 

file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_Based (userID, sessionID, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [40]:
# Examples
query = """
SELECT artist, song, firstName, lastName
FROM user_session_Based
WHERE userId=10 AND sessionId=182
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r)


Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## song_listened_by_users_Based (Question 3)
### Example Query:
> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.
### Partition Key
- `song` is used for Parition Key so that the table is distributed based on song.
### Primary Key
- `song` column itself will not be unique, because many users will listen to the same song throughout the history.
- in order to provide the unique for the PK, the `userid` is also added.
> 

In [41]:
# Support Query Example 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_listened_by_users_Based"
query += """
(
    song text,
    userid text,
    firstName text,
    lastName text,
    PRIMARY KEY(song, userid)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)


In [42]:
# Load Data into song_listened_by_users_based Table
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO song_listened_by_users_Based (song, userid, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, ( line[9], line[10], line[1], line[4]))


In [43]:
# Example
query ="""
SELECT firstName, lastName, song
FROM song_listened_by_users_Based
WHERE song='All Hands Against His Own'
"""
try:
    rows=session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r)

Row(firstname='Jacqueline', lastname='Lynch', song='All Hands Against His Own')
Row(firstname='Tegan', lastname='Levine', song='All Hands Against His Own')
Row(firstname='Sara', lastname='Johnson', song='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [44]:
query1 = "DROP TABLE IF EXISTS session_tracking_Based"
query2 = "DROP TABLE IF EXISTS user_session_Based"
query3 = "DROP TABLE IF EXISTS song_listened_by_users_Based"
queries = [query1, query2, query3]
try:
    for q in queries:
        session.execute(q)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [45]:
session.shutdown()
cluster.shutdown()